In [ ]:
# Step 1: Setup and Dependencies
# Install required packages
%pip install pdfplumber pypdf openai faiss-cpu numpy python-dotenv pdf2image pillow

# Import libraries
import os
import json
import base64
from pathlib import Path
from typing import List, Dict, Any
import numpy as np
import pdfplumber
from concurrent.futures import ThreadPoolExecutor, as_completed
from pdf2image import convert_from_path
from PIL import Image
from dotenv import load_dotenv
from openai import OpenAI
import faiss

# Load environment variables
load_dotenv()

# Initialize OpenAI client
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

client = OpenAI(api_key=api_key)

print("Setup complete! OpenAI client initialized.")

Note: you may need to restart the kernel to use updated packages.
Setup complete! OpenAI client initialized.


In [2]:
# Step 2: Check PDF for Embedded Text
def check_pdf_has_text(pdf_path: str) -> bool:
    """Check if PDF contains extractable text."""
    try:
        # Try with pdfplumber first (more reliable)
        with pdfplumber.open(pdf_path) as pdf:
            # Check first few pages
            for page_num in range(min(5, len(pdf.pages))):
                page = pdf.pages[page_num]
                text = page.extract_text()
                if text and len(text.strip()) > 100:  # Has meaningful text
                    return True
        return False
    except Exception as e:
        print(f"Error checking PDF: {e}")
        return False

# Check your PDF
pdf_path = "pdfs/salesforce_release_notes_1-19-2026.pdf"
has_text = check_pdf_has_text(pdf_path)
print(f"PDF has extractable text: {has_text}")

if has_text:
    print("✓ Using direct text extraction (much faster!)")
else:
    print("⚠ PDF doesn't have extractable text - will use image OCR fallback")

PDF has extractable text: True
✓ Using direct text extraction (much faster!)


In [3]:
# Step 3: Extract Text with Structure
def extract_structured_text(pdf_path: str) -> List[Dict[str, Any]]:
    """
    Extract text from PDF preserving structure (headings, sections).
    Returns list of structured chunks.
    """
    chunks = []
    
    with pdfplumber.open(pdf_path) as pdf:
        print(f"Processing {len(pdf.pages)} pages...")
        
        for page_num, page in enumerate(pdf.pages, start=1):
            # Extract text with layout preservation
            text = page.extract_text(layout=True)
            
            if not text or len(text.strip()) < 50:
                continue
            
            # Split into paragraphs/sections
            paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
            
            # Filter out headers/footers
            filtered_paragraphs = []
            for para in paragraphs:
                para_lower = para.lower()
                # Skip common header/footer patterns (only if short)
                if any(skip in para_lower for skip in [
                    'salesforce release notes', 'page', 'copyright',
                    'confidential', 'table of contents'
                ]) and len(para) < 100:  # Only skip if it's a short header/footer
                    continue
                filtered_paragraphs.append(para)
            
            # Create chunks from paragraphs
            # Group consecutive paragraphs into chunks (500-2000 chars each)
            current_chunk_text = []
            current_chunk_size = 0
            
            for para in filtered_paragraphs:
                para_size = len(para)
                
                # If adding this para would exceed 2000 chars, save current chunk
                if current_chunk_size + para_size > 2000 and current_chunk_text:
                    chunk_text = "\n\n".join(current_chunk_text)
                    chunks.append({
                        "page_number": page_num,
                        "feature_name": current_chunk_text[0][:100] if current_chunk_text else "Untitled",
                        "content": chunk_text,
                        "text": chunk_text
                    })
                    current_chunk_text = [para]
                    current_chunk_size = para_size
                else:
                    current_chunk_text.append(para)
                    current_chunk_size += para_size
            
            # Save last chunk on page if it exists
            if current_chunk_text:
                chunk_text = "\n\n".join(current_chunk_text)
                chunks.append({
                    "page_number": page_num,
                    "feature_name": current_chunk_text[0][:100] if current_chunk_text else "Untitled",
                    "content": chunk_text,
                    "text": chunk_text
                })
    
    print(f"Created {len(chunks)} text chunks from {len(pdf.pages)} pages")
    return chunks

# Extract text if PDF has text
if has_text:
    print("Extracting text with structure...")
    text_chunks = extract_structured_text(pdf_path)
    print(f"Extracted {len(text_chunks)} text chunks")
else:
    print("Skipping text extraction - will use image processing")
    text_chunks = []

# Step 4: GPT Processing for Structured Extraction
def extract_release_notes_from_text(chunk: Dict[str, Any], client: OpenAI) -> Dict[str, Any]:
    """
    Extract structured release notes from text chunk using GPT-4o-mini.
    Much faster than image processing!
    """
    system_prompt = """You are an expert at extracting Salesforce release notes from text.
Extract structured information from the provided text chunk.

CRITICAL: Do NOT extract content from index pages, table of contents, or navigation pages.
These pages typically contain:
- Section headings with brief category descriptions (e.g., "Analytics", "Commerce", "Customization")
- "How to use this document" type instructions
- General overview text without specific feature details
- Lists of product categories with one-sentence summaries
- Pages that say things like "Learn about...", "Read about...", "Includes features for..."
- Pages with multiple category headings but no detailed feature descriptions

ONLY extract actual release notes that describe specific features, enhancements, or changes with detailed information.
A real release note should have:
- A specific feature name (not just a category name like "Analytics" or "Commerce")
- A detailed description explaining what the feature does or what changed
- Specific technical details, capabilities, or user-facing changes

If the text appears to be an index, table of contents, or navigation page with only category summaries, return an empty release_notes array.

For each release note, extract ALL of the following information if present:
- feature_name: The name of the feature or change
- category: High-level category (e.g., Sales Cloud, Service Cloud, Platform, API, Security, etc.)
- product_area: Specific product/technology area (e.g., Flow Builder, Lightning Web Components, Apex, REST API, SOQL, Mobile, etc.)
- change_type: Type of change (e.g., "New Feature", "Enhancement", "Breaking Change", "Deprecation", "Retirement", "Security Update", "Bug Fix", "Stabilization")
- description: Detailed description of the feature or change (include all important details)
- availability: When it's available (e.g., "Available Now", "Pilot", "Beta", specific date)
- feature_status: Status/impact level (e.g., "Beta", "Pilot", "General Availability", "Preview", "Release Update")
- activation_required: How to activate (e.g., "Auto-enabled", "Requires Release Update", "Manual Activation", "Opt-in", "N/A")
- edition: Which Salesforce edition(s) it applies to (if mentioned)
- api_version: Any API version numbers mentioned (e.g., "API 59.0", "API Version 60.0", "LWC API Version 59.0")
- timeline: Important dates (activation date, preview period, sunset date, etc.)
- affected_components: List of specific components, classes, modules, or features mentioned
- breaking_changes: Details about breaking changes, migration requirements, or compatibility notes (if applicable)
- security_notes: Any security-related information or requirements

Return a JSON object with this structure:
{
    "page_number": <page number>,
    "release_notes": [
        {
            "feature_name": "...",
            "category": "...",
            "product_area": "...",
            "change_type": "...",
            "description": "...",
            "availability": "...",
            "feature_status": "...",
            "activation_required": "...",
            "edition": "...",
            "api_version": "...",
            "timeline": "...",
            "affected_components": ["..."],
            "breaking_changes": "...",
            "security_notes": "..."
        }
    ]
}

IMPORTANT EXTRACTION GUIDELINES:
1. Extract EVERY release note in the text, even if some fields are empty
2. If a field is not mentioned, use "N/A" or empty string/array as appropriate
3. Pay special attention to:
   - Breaking changes and deprecations (these are critical)
   - API version numbers (especially for LWC and Apex)
   - Release update requirements
   - Security-related changes
4. For "affected_components", extract as an array of strings
5. Include full descriptions - don't truncate important details
6. DO NOT extract category summaries or index page content - only extract actual feature release notes

If there are no release notes in the text (e.g., it's an index page), return an empty release_notes array."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": f"Extract release notes from this text:\n\n{chunk['text']}"
                }
            ],
            response_format={"type": "json_object"},
            temperature=0.1
        )
        
        result = json.loads(response.choices[0].message.content)
        result["page_number"] = chunk.get("page_number", 0)
        return result
        
    except Exception as e:
        print(f"Error processing chunk from page {chunk.get('page_number', 'unknown')}: {str(e)}")
        return {"page_number": chunk.get("page_number", 0), "release_notes": []}

# Fallback: Image processing functions (kept for compatibility)
def encode_image_to_base64(image_path: str) -> str:
    """Encode image to base64 string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def extract_release_notes(image_path: str, client: OpenAI) -> Dict[str, Any]:
    """
    Extract release notes from an image using GPT-4o-mini.
    
    Returns structured JSON with release notes.
    """
    # Encode image to base64
    base64_image = encode_image_to_base64(image_path)
    
    # System prompt for structured extraction
    system_prompt = """You are an expert at extracting Salesforce release notes from PDF pages. 
Extract all release notes from the image and return them in a structured JSON format.

For each release note, extract ALL of the following information if present:
- feature_name: The name of the feature or change
- category: High-level category (e.g., Sales Cloud, Service Cloud, Platform, API, Security, etc.)
- product_area: Specific product/technology area (e.g., Flow Builder, Lightning Web Components, Apex, REST API, SOQL, Mobile, etc.)
- change_type: Type of change (e.g., "New Feature", "Enhancement", "Breaking Change", "Deprecation", "Retirement", "Security Update", "Bug Fix", "Stabilization")
- description: Detailed description of the feature or change (include all important details)
- availability: When it's available (e.g., "Available Now", "Pilot", "Beta", specific date)
- feature_status: Status/impact level (e.g., "Beta", "Pilot", "General Availability", "Preview", "Release Update")
- activation_required: How to activate (e.g., "Auto-enabled", "Requires Release Update", "Manual Activation", "Opt-in", "N/A")
- edition: Which Salesforce edition(s) it applies to (if mentioned)
- api_version: Any API version numbers mentioned (e.g., "API 59.0", "API Version 60.0", "LWC API Version 59.0")
- timeline: Important dates (activation date, preview period, sunset date, etc.)
- affected_components: List of specific components, classes, modules, or features mentioned
- breaking_changes: Details about breaking changes, migration requirements, or compatibility notes (if applicable)
- security_notes: Any security-related information or requirements

Return a JSON object with this structure:
{
    "page_number": <page number>,
    "release_notes": [
        {
            "feature_name": "...",
            "category": "...",
            "product_area": "...",
            "change_type": "...",
            "description": "...",
            "availability": "...",
            "feature_status": "...",
            "activation_required": "...",
            "edition": "...",
            "api_version": "...",
            "timeline": "...",
            "affected_components": ["..."],
            "breaking_changes": "...",
            "security_notes": "..."
        }
    ]
}

IMPORTANT EXTRACTION GUIDELINES:
1. Extract EVERY release note on the page, even if some fields are empty
2. If a field is not mentioned, use "N/A" or empty string/array as appropriate
3. Pay special attention to:
   - Breaking changes and deprecations (these are critical)
   - API version numbers (especially for LWC and Apex)
   - Release update requirements
   - Security-related changes
4. For "affected_components", extract as an array of strings
5. Include full descriptions - don't truncate important details
6. Skip headers, footers, page numbers, and navigation elements
7. If multiple related features are grouped together, extract them as separate notes if they have distinct names/descriptions

If there are no release notes on the page, return an empty release_notes array.
Only extract actual release notes, skip headers, footers, and navigation elements."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Extract all release notes from this page in the specified JSON format."
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            response_format={"type": "json_object"},
            temperature=0.1
        )
        
        # Parse JSON response
        result = json.loads(response.choices[0].message.content)
        return result
        
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return {"page_number": 0, "release_notes": []}


Extracting text with structure...
Processing 842 pages...
Created 841 text chunks from 842 pages
Extracted 841 text chunks


In [4]:
# Step 5: Process Text Chunks in Parallel (or fallback to images)
if has_text and text_chunks:
    print("Processing text chunks with GPT-4o-mini in parallel...")
    print(f"Total chunks to process: {len(text_chunks)}")
    
    # Process chunks in parallel
    def process_chunk_with_progress(chunk_idx_chunk):
        idx, chunk = chunk_idx_chunk
        if (idx + 1) % 50 == 0:
            print(f"  Processing chunk {idx + 1}/{len(text_chunks)}...")
        return extract_release_notes_from_text(chunk, client)
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        all_extracted_data = list(executor.map(
            process_chunk_with_progress,
            enumerate(text_chunks)
        ))
    
    print(f"\nProcessed {len(all_extracted_data)} chunks")
    
else:
    # Fallback: Image processing
    print("Using image processing fallback...")
    from pdf2image import convert_from_path
    
    def convert_pdf_to_images(pdf_path: str, output_folder: str = "images") -> List[str]:
        Path(output_folder).mkdir(parents=True, exist_ok=True)
        pdf_name = Path(pdf_path).stem
        images = convert_from_path(pdf_path)
        image_paths = []
        for page_num, image in enumerate(images, start=1):
            image_filename = f"{pdf_name}_page_{page_num}.jpg"
            image_path = os.path.join(output_folder, image_filename)
            image.save(image_path, "JPEG")
            image_paths.append(image_path)
        return image_paths
    
    def process_all_images_parallel(image_paths: List[str], client: OpenAI, max_workers: int = 5) -> List[Dict[str, Any]]:
        all_notes = []
        from concurrent.futures import as_completed
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_path = {
                executor.submit(extract_release_notes, path, client): path
                for path in image_paths
            }
            for future in as_completed(future_to_path):
                try:
                    result = future.result()
                    all_notes.append(result)
                except Exception as e:
                    print(f"Error: {str(e)}")
                    all_notes.append({"page_number": 0, "release_notes": []})
        all_notes.sort(key=lambda x: x.get("page_number", 0))
        return all_notes
    
    image_paths = convert_pdf_to_images(pdf_path)
    all_extracted_data = process_all_images_parallel(image_paths, client, max_workers=5)

# Filter out pages with no valid release notes
filtered_data = []
for page_data in all_extracted_data:
    notes = page_data.get("release_notes", [])
    # Filter: keep only pages with at least one note that has meaningful content
    valid_notes = []
    for note in notes:
        feature_name = note.get("feature_name", "").strip()
        description = note.get("description", "").strip()
        
        # Basic validation
        if not feature_name or not description:
            continue
        
        # Filter out category/section descriptions (common in index pages)
        if len(description) < 50:  # Too short to be a real feature description
            continue
        
        # Filter out notes that look like section headers
        skip_patterns = [
            "learn about", "read about", "includes features for", 
            "enhancements include", "provides", "offers",
            "check back", "see", "review", "find"
        ]
        description_lower = description.lower()
        if any(pattern in description_lower[:100] for pattern in skip_patterns):
            if len(description) < 150:
                continue
        
        valid_notes.append(note)
    
    if valid_notes:
        page_data["release_notes"] = valid_notes
        filtered_data.append(page_data)

print(f"\nExtraction complete!")
print(f"Total chunks/pages processed: {len(all_extracted_data)}")
print(f"Pages with valid release notes: {len(filtered_data)}")

# Count total notes
total_notes = sum(len(page.get("release_notes", [])) for page in filtered_data)
print(f"Total release notes extracted: {total_notes}")

# Display sample in markdown format for readability
if filtered_data:
    print("\n--- Sample Release Note (Markdown Format) ---")
    sample_page = filtered_data[0]
    print(f"Page {sample_page.get('page_number', 'N/A')}:")
    for note in sample_page.get("release_notes", [])[:2]:  # Show first 2 notes
        print(f"\n**{note.get('feature_name', 'N/A')}**")
        print(f"Category: {note.get('category', 'N/A')}")
        print(f"Product Area: {note.get('product_area', 'N/A')}")
        print(f"Change Type: {note.get('change_type', 'N/A')}")
        print(f"Description: {note.get('description', 'N/A')[:200]}...")
        if note.get('breaking_changes') and note.get('breaking_changes') != 'N/A':
            print(f"⚠️ Breaking Changes: {note.get('breaking_changes', 'N/A')[:100]}...")

Processing text chunks with GPT-4o-mini in parallel...
Total chunks to process: 841
  Processing chunk 50/841...
  Processing chunk 100/841...
  Processing chunk 150/841...
  Processing chunk 200/841...
  Processing chunk 250/841...
  Processing chunk 300/841...
  Processing chunk 350/841...
  Processing chunk 400/841...
  Processing chunk 450/841...
  Processing chunk 500/841...
  Processing chunk 550/841...
  Processing chunk 600/841...
  Processing chunk 650/841...
  Processing chunk 700/841...
  Processing chunk 750/841...
  Processing chunk 800/841...
Error processing chunk from page 762: Connection error.

Processed 841 chunks

Extraction complete!
Total chunks/pages processed: 841
Pages with valid release notes: 783
Total release notes extracted: 3265

--- Sample Release Note (Markdown Format) ---
Page 6:

**Setup with Agentforce (beta)**
Category: Customization
Product Area: Setup
Change Type: New Feature
Description: Complete Setup tasks more efficiently with the help of an AI

In [11]:
def process_all_images(image_paths: List[str], client: OpenAI) -> List[Dict[str, Any]]:
    """Process all images and extract release notes."""
    all_notes = []
    
    for idx, image_path in enumerate(image_paths, start=1):
        print(f"Processing page {idx}/{len(image_paths)}: {os.path.basename(image_path)}")
        result = extract_release_notes(image_path, client)
        
        # Add page number if not present
        if "page_number" not in result:
            result["page_number"] = idx
        
        # Add image path for reference
        result["image_path"] = image_path
        
        all_notes.append(result)
        
        # Print summary
        num_notes = len(result.get("release_notes", []))
        print(f"  Extracted {num_notes} release note(s)")
    
    return all_notes

In [9]:
# NOTE: Processing is handled in Cell 3 above
# This cell is intentionally left empty - all extraction and filtering happens in Cell 3
# which handles both text extraction (if PDF has text) and image processing (fallback)

In [13]:
# Step 4: Embedding Generation
# Ensure as_completed is imported
from concurrent.futures import as_completed

# Save filtered release notes to JSON
with open("filtered_release_notes.json", "w") as f:
    json.dump(filtered_data, f, indent=2)

print("Saved filtered release notes to filtered_release_notes.json")

def generate_embeddings(notes_data: List[Dict[str, Any]], client: OpenAI, max_workers: int = 10) -> tuple:
    """
    Generate embeddings for release notes using text-embedding-3-large with parallel processing.
    
    Args:
        notes_data: List of page data with release notes
        client: OpenAI client
        max_workers: Number of parallel workers (default: 10)
    
    Returns:
        Tuple of (numpy array of embeddings, list of note items)
    """
    # Flatten all notes into a list
    all_notes = []
    for page_data in notes_data:
        for note in page_data.get("release_notes", []):
            # Combine multiple fields for richer embedding
            text_parts = [
                note.get('feature_name', ''),
                note.get('category', ''),
                note.get('product_area', ''),
                note.get('change_type', ''),
                note.get('description', ''),
                note.get('availability', ''),
                note.get('feature_status', ''),
                note.get('api_version', ''),
                note.get('timeline', ''),
            ]
            # Add breaking changes and security notes if present
            if note.get('breaking_changes') and note.get('breaking_changes') != 'N/A':
                text_parts.append(f"Breaking Changes: {note.get('breaking_changes')}")
            if note.get('security_notes') and note.get('security_notes') != 'N/A':
                text_parts.append(f"Security: {note.get('security_notes')}")
            # Add affected components if present
            if note.get('affected_components') and note.get('affected_components') != []:
                components_str = ', '.join(note.get('affected_components', []))
                text_parts.append(f"Affected Components: {components_str}")
            
            text_to_embed = " ".join([part for part in text_parts if part and part != 'N/A']).strip()
            if text_to_embed:
                all_notes.append({
                    "text": text_to_embed,
                    "note": note,
                    "page_data": page_data
                })
    
    print(f"Generating embeddings for {len(all_notes)} release notes (parallel processing with {max_workers} workers)...")
    
    # Function to generate a single embedding
    def generate_single_embedding(note_item):
        try:
            response = client.embeddings.create(
                model="text-embedding-3-large",
                input=note_item["text"]
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"Error embedding note: {str(e)}")
            # Return zero vector as fallback
            return [0.0] * 3072
    
    # Generate embeddings in parallel
    embeddings = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_idx = {
            executor.submit(generate_single_embedding, note_item): idx
            for idx, note_item in enumerate(all_notes)
        }
        
        # Collect results as they complete (maintain order)
        results = [None] * len(all_notes)
        completed = 0
        for future in as_completed(future_to_idx):
            idx = future_to_idx[future]
            try:
                results[idx] = future.result()
                completed += 1
                if completed % 50 == 0:
                    print(f"  Processed {completed}/{len(all_notes)} embeddings...")
            except Exception as e:
                print(f"Error getting embedding result: {str(e)}")
                results[idx] = [0.0] * 3072
        
        embeddings = results
    
    # Convert to numpy array
    embeddings_array = np.array(embeddings, dtype=np.float32)
    
    return embeddings_array, all_notes

# Generate embeddings
embeddings_matrix, notes_list = generate_embeddings(filtered_data, client)

# Verify embedding matrix shape
print(f"\nEmbedding matrix shape: {embeddings_matrix.shape}")
print(f"Expected shape: (n_notes, 3072) for text-embedding-3-large")
print(f"Data type: {embeddings_matrix.dtype}")

# Display sample values
print(f"\nSample embedding (first 10 values): {embeddings_matrix[0][:10]}")
print(f"Embedding min value: {embeddings_matrix.min():.6f}")
print(f"Embedding max value: {embeddings_matrix.max():.6f}")
print(f"Embedding mean value: {embeddings_matrix.mean():.6f}")

print(f"\nEmbedding generation complete! Generated {len(embeddings_matrix)} embeddings.")

Saved filtered release notes to filtered_release_notes.json
Generating embeddings for 3265 release notes (parallel processing with 10 workers)...
  Processed 50/3265 embeddings...
  Processed 100/3265 embeddings...
  Processed 150/3265 embeddings...
  Processed 200/3265 embeddings...
  Processed 250/3265 embeddings...
  Processed 300/3265 embeddings...
  Processed 350/3265 embeddings...
  Processed 400/3265 embeddings...
  Processed 450/3265 embeddings...
  Processed 500/3265 embeddings...
  Processed 550/3265 embeddings...
  Processed 600/3265 embeddings...
  Processed 650/3265 embeddings...
  Processed 700/3265 embeddings...
  Processed 750/3265 embeddings...
  Processed 800/3265 embeddings...
  Processed 850/3265 embeddings...
  Processed 900/3265 embeddings...
  Processed 950/3265 embeddings...
  Processed 1000/3265 embeddings...
  Processed 1050/3265 embeddings...
  Processed 1100/3265 embeddings...
  Processed 1150/3265 embeddings...
  Processed 1200/3265 embeddings...
  Processe

In [14]:
# Step 5: FAISS Index Creation
# Get embedding dimension
embedding_dim = embeddings_matrix.shape[1]
print(f"Creating FAISS index with dimension: {embedding_dim}")

# Create FAISS index (L2 distance)
index = faiss.IndexFlatL2(embedding_dim)

# Add vectors to the index
print(f"Adding {len(embeddings_matrix)} vectors to index...")
index.add(embeddings_matrix)

# Verify index contains correct number of vectors
print(f"Index contains {index.ntotal} vectors")
assert index.ntotal == len(embeddings_matrix), "Index vector count mismatch!"

# Save index
index_filename = "filtered_sf_release_notes_index.index"
faiss.write_index(index, index_filename)
print(f"Index saved to {index_filename}")

# Create metadata file
metadata = {
    "notes": [note_item["note"] for note_item in notes_list],
    "image_paths": [note_item["page_data"].get("image_path", "") for note_item in notes_list]
}

metadata_filename = "metadata_json.json"
with open(metadata_filename, "w") as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata saved to {metadata_filename}")
print(f"Metadata contains {len(metadata['notes'])} notes and {len(metadata['image_paths'])} image paths")

print("\nFAISS index creation complete!")

Creating FAISS index with dimension: 3072
Adding 3265 vectors to index...
Index contains 3265 vectors
Index saved to filtered_sf_release_notes_index.index
Metadata saved to metadata_json.json
Metadata contains 3265 notes and 3265 image paths

FAISS index creation complete!


In [15]:
# Step 6: Query Function
def query_embeddings(query: str, index: faiss.Index, metadata: Dict[str, Any], 
                     client: OpenAI, k: int = 5) -> List[Dict[str, Any]]:
    """
    Generate embedding for query and search FAISS index for top-k similar notes.
    
    Args:
        query: User query string
        index: FAISS index
        metadata: Metadata dictionary with 'notes' and 'image_paths'
        client: OpenAI client
        k: Number of top results to return (default: 5)
    
    Returns:
        List of dictionaries containing note information, distances, and indices
    """
    # Generate embedding for query
    print(f"Generating embedding for query: '{query}'")
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=query
    )
    query_embedding = np.array([response.data[0].embedding], dtype=np.float32)
    
    # Search FAISS index
    print(f"Searching index for top-{k} results...")
    distances, indices = index.search(query_embedding, k)
    
    # Extract top-k similarities
    results = []
    for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
        if idx < len(metadata["notes"]):
            result = {
                "rank": i + 1,
                "distance": float(distance),
                "index": int(idx),
                "note": metadata["notes"][idx],
                "image_path": metadata["image_paths"][idx] if idx < len(metadata["image_paths"]) else ""
            }
            results.append(result)
    
    print(f"Found {len(results)} results")
    return results

# Test the query function
test_query = "What are the new features in Sales Cloud?"
print("Testing query function...")
test_results = query_embeddings(test_query, index, metadata, client, k=5)

print("\n--- Sample Query Results ---")
for result in test_results[:3]:  # Show first 3
    print(f"\nRank {result['rank']} (Distance: {result['distance']:.4f})")
    print(f"Feature: {result['note'].get('feature_name', 'N/A')}")
    print(f"Category: {result['note'].get('category', 'N/A')}")
    print(f"Change Type: {result['note'].get('change_type', 'N/A')}")
    print(f"Description: {result['note'].get('description', 'N/A')[:100]}...")
    if result['note'].get('breaking_changes') and result['note'].get('breaking_changes') != 'N/A':
        print(f"⚠️ Breaking Changes: {result['note'].get('breaking_changes', 'N/A')[:80]}...")

Testing query function...
Generating embedding for query: 'What are the new features in Sales Cloud?'
Searching index for top-5 results...
Found 5 results

--- Sample Query Results ---

Rank 1 (Distance: 0.8084)
Feature: Revenue Cloud for Communications on Salesforce Platform
Category: Industries
Change Type: New Feature
Description: Discover, set up, and configure Revenue Cloud for Communications features with Salesforce Go. Boost ...

Rank 2 (Distance: 0.8220)
Feature: AI-Powered Selling Initial Setup
Category: Sales
Change Type: New Feature
Description: AI-Powered Selling Initial Setup simplifies configuration for new and existing orgs. It automaticall...

Rank 3 (Distance: 0.8383)
Feature: Get Your Agentic Sales Org Ready in a Few Clicks
Category: Sales Cloud
Change Type: New Feature
Description: This feature simplifies the process of preparing your Agentic Sales Organization with just a few cli...


In [16]:
# Step 7: RAG Pipeline
def combine_results_for_gpt(results: List[Dict[str, Any]]) -> str:
    """
    Combine all retrieved notes into a formatted string for GPT context.
    
    Args:
        results: List of result dictionaries from query_embeddings
    
    Returns:
        Formatted string with all notes
    """
    context_parts = []
    for result in results:
        note = result["note"]
        context = f"Feature: {note.get('feature_name', 'N/A')}\n"
        context += f"Category: {note.get('category', 'N/A')}\n"
        context += f"Product Area: {note.get('product_area', 'N/A')}\n"
        context += f"Change Type: {note.get('change_type', 'N/A')}\n"
        context += f"Description: {note.get('description', 'N/A')}\n"
        context += f"Availability: {note.get('availability', 'N/A')}\n"
        context += f"Feature Status: {note.get('feature_status', 'N/A')}\n"
        context += f"Activation Required: {note.get('activation_required', 'N/A')}\n"
        context += f"Edition: {note.get('edition', 'N/A')}\n"
        context += f"API Version: {note.get('api_version', 'N/A')}\n"
        context += f"Timeline: {note.get('timeline', 'N/A')}\n"
        if note.get('affected_components'):
            context += f"Affected Components: {', '.join(note.get('affected_components', []))}\n"
        if note.get('breaking_changes') and note.get('breaking_changes') != 'N/A':
            context += f"⚠️ Breaking Changes: {note.get('breaking_changes', 'N/A')}\n"
        if note.get('security_notes') and note.get('security_notes') != 'N/A':
            context += f"🔒 Security Notes: {note.get('security_notes', 'N/A')}\n"
        context += "---"
        context_parts.append(context)
    return "\n\n".join(context_parts)

def create_gpt_prompt(query: str, context: str, system_prompt: str = None) -> Dict[str, str]:
    """
    Create system and user prompts for GPT.
    
    Args:
        query: User query
        context: Combined context from retrieved notes
        system_prompt: Optional custom system prompt
    
    Returns:
        Dictionary with 'system' and 'user' keys
    """
    if system_prompt is None:
        system_prompt = """You are a helpful assistant that answers questions about Salesforce release notes.
Use the provided release notes context to answer the user's question accurately.
If the context doesn't contain relevant information, say so.
Be concise but comprehensive in your answers."""

    user_prompt = f"""Based on the following Salesforce release notes, please answer the question.

Release Notes Context:
{context}

Question: {query}

Please provide a clear and helpful answer based on the release notes above."""

    return {
        "system": system_prompt,
        "user": user_prompt
    }

def rag_query(user_query: str, index: faiss.Index, metadata: Dict[str, Any], 
              client: OpenAI, k: int = 5, model: str = "gpt-4o-mini", 
              system_prompt: str = None) -> str:
    """
    Complete RAG pipeline: query embedding, retrieval, and GPT response.
    
    Args:
        user_query: User's question
        index: FAISS index
        metadata: Metadata dictionary
        client: OpenAI client
        k: Number of top results to retrieve (default: 5)
        model: GPT model to use (default: "gpt-4o-mini")
        system_prompt: Optional custom system prompt
    
    Returns:
        GPT's response as a string
    """
    # Step 1: Query embeddings and retrieve top-k results
    results = query_embeddings(user_query, index, metadata, client, k)
    
    if not results:
        return "No relevant release notes found for your query."
    
    # Step 2: Combine results for GPT
    context = combine_results_for_gpt(results)
    
    # Step 3: Create GPT prompt
    prompts = create_gpt_prompt(user_query, context, system_prompt)
    
    # Step 4: Send to GPT API
    print(f"\nSending query to {model}...")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompts["system"]},
            {"role": "user", "content": prompts["user"]}
        ],
        temperature=0.3
    )
    
    # Extract and return GPT's response
    gpt_response = response.choices[0].message.content
    return gpt_response

# Test the RAG pipeline
print("Testing RAG pipeline...")
test_rag_query = "What are the new features in Sales Cloud?"
rag_response = rag_query(test_rag_query, index, metadata, client, k=5)

print("\n" + "="*60)
print("RAG Query Result:")
print("="*60)
print(rag_response)
print("="*60)

Testing RAG pipeline...
Generating embedding for query: 'What are the new features in Sales Cloud?'
Searching index for top-5 results...
Found 5 results

Sending query to gpt-4o-mini...

RAG Query Result:
The new features in Sales Cloud based on the release notes are:

1. **AI-Powered Selling Initial Setup**: This feature simplifies the configuration for new and existing organizations by automatically turning on base Sales features and agents, and assigning the correct permissions to sellers. It provides a consistent, ready-to-use sales environment immediately. Availability is set for December 2025, and it requires manual activation.

2. **Sales Workspace**: This feature offers sellers a consolidated workspace that provides clarity on goals, key metrics, AI-driven recommendations, and actionable insights. It replaces the traditional Seller Home to enhance the daily selling experience. Like the AI-Powered Selling Initial Setup, it will be available in December 2025 and requires manual a

In [18]:
# Step 8: Demo - Load Index and Run Sample Queries
# Load the saved FAISS index and metadata
print("Loading saved index and metadata...")
loaded_index = faiss.read_index("filtered_sf_release_notes_index.index")
print(f"Loaded index with {loaded_index.ntotal} vectors")

with open("metadata_json.json", "r") as f:
    loaded_metadata = json.load(f)
print(f"Loaded metadata with {len(loaded_metadata['notes'])} notes")

# Sample queries to test the complete pipeline
sample_queries = [
    "What changes were made to Flow Builder?",
    "What are the new features in Sales Cloud?",
    "Are there any API changes in this release?",
    "What security features were added?",
    "What improvements were made to reporting?"
]

print("\n" + "="*80)
print("Running Sample Queries")
print("="*80)

for i, query in enumerate(sample_queries, 1):
    print(f"\n{'='*80}")
    print(f"Query {i}: {query}")
    print(f"{'='*80}")
    
    response = rag_query(query, loaded_index, loaded_metadata, client, k=5)
    print(f"\nAnswer:\n{response}")
    print("\n" + "-"*80)

Loading saved index and metadata...
Loaded index with 3265 vectors
Loaded metadata with 3265 notes

Running Sample Queries

Query 1: What changes were made to Flow Builder?
Generating embedding for query: 'What changes were made to Flow Builder?'
Searching index for top-5 results...
Found 5 results

Sending query to gpt-4o-mini...

Answer:
The recent updates to Flow Builder include several enhancements:

1. **Natural Language Changes**: Users can now make changes to record-triggered and scheduled-triggered flows using natural language.

2. **Navigation Improvements**: The Flow Builder canvas has become easier to navigate with collapsible branching paths and more intuitive mouse scrolling controls.

3. **Agentforce Panel Access**: Users can access the Agentforce panel without requiring admin setup.

4. **Data Table Screen Component**: Enhancements allow for sorting and editing records within the Data Table screen component.

5. **Screen Flow Customization**: Users can customize screen f